In [1]:
from io_utils import read_fbin, read_ibin
import faiss
print(faiss.__version__)
import numpy as np
file_path = "/root/code/vectordbindexing/Text2Image/base.1M.fbin"
query_path = "/root/code/vectordbindexing/Text2Image/query.public.100K.fbin"
ground_truth_path = "/root/code/vectordbindexing/Text2Image/groundtruth.public.100K.ibin"

# read datasets
print("\n\nreading image vector: ---")
data_vector = read_fbin(file_path)
print(type(data_vector))
print(data_vector.ndim, data_vector.shape, data_vector.dtype, data_vector.size)
# print(data_vector[:1])  # Print first 1 elements to verify content

train_data_vector = data_vector[:100000]
insert_1_percent = data_vector[500000:505000]
insert_2_percent = data_vector[505000:510000]
insert_3_percent = data_vector[510000:515000]
insert_4_percent = data_vector[515000:520000]
insert_5_percent = data_vector[520000:525000]
insert_10_percent = data_vector[525000:550000]

# read querys
print("\n\nreading querys: ---")
query_vector = read_fbin(query_path)
print(type(query_vector))
print(query_vector.ndim, query_vector.shape, query_vector.dtype, query_vector.size)
# print(query_vector[0])  # Print first 3 elements to verify content

1.11.0


reading image vector: ---
<class 'numpy.ndarray'>
2 (1000000, 200) float32 200000000


reading querys: ---
<class 'numpy.ndarray'>
2 (100000, 200) float32 20000000


In [2]:
import hnsw_cosine_norm as hnsw_cosine
import simple_sim_hash
import importlib
importlib.reload(hnsw_cosine)

# 创建数据预处理器
print("\n=== 创建数据预处理器 ===")
# 模拟text和image数据（实际应用中需要根据实际情况分割）
n_image = len(train_data_vector)

text_data = query_vector
image_data = train_data_vector

print(f"分割数据: {len(text_data)} text, {len(image_data)} image")

# 创建预处理器
preprocessor = hnsw_cosine.DataPreprocessor(
    use_pca=True,
    n_components=128,  # 降维到128维
    use_global_whitening=True,
    sub_modality_scaling=True
)

print("拟合预处理器...")
preprocessor.fit(text_data, image_data, sample_size=10000)

# 独立处理embedding：先统一处理所有数据
print("\n=== 独立处理embedding ===")
print("处理text数据...")
processed_text_data = preprocessor.transform_batch(text_data, "text")
print(f"Text数据预处理完成: {processed_text_data.shape}")

print("处理image数据...")
processed_image_data = preprocessor.transform_batch(image_data, "image")
print(f"Image数据预处理完成: {processed_image_data.shape}")

# 创建不带预处理器的索引（因为数据已经预处理过了）
print("\n=== 创建HNSW索引 ===")
index = hnsw_cosine.HNSWIndex(M=64, ef_construction=128, ef_search=64, random_seed=1, preprocessor=None)
simHash = simple_sim_hash.SimpleSimHash(dim=128)  # 降维后的维度

IMAGE_IDX_SET = set()

# 使用批量添加方法加速构建过程
print("批量添加预处理后的text数据到索引...")
import time
start_time = time.time()

# 准备text数据的ID和模态信息
text_ids = list(range(len(processed_text_data)))
text_modalities = ["text"] * len(processed_text_data)
text_original_ids = list(range(len(processed_text_data)))

# 批量添加text数据（使用较小的批次大小）
added_text_ids = index.add_items_batch(
    vectors=processed_text_data,
    ids=text_ids,
    modalities=text_modalities,
    original_ids=text_original_ids,
    preprocessed=True,
    batch_size=500  # 使用较小的批次大小
)

# 更新IMAGE_IDX_SET
for img_id in added_text_ids:
    IMAGE_IDX_SET.add(img_id)

n_text = len(processed_text_data)
text_time = time.time() - start_time
print(f"  已添加 {len(added_text_ids)} 个text向量，耗时: {text_time:.2f}秒")

# 准备image数据的ID和模态信息
print("批量添加预处理后的image数据到索引...")
image_start_time = time.time()

image_ids = list(range(n_text, n_text + len(processed_image_data)))
image_modalities = ["image"] * len(processed_image_data)
image_original_ids = list(range(len(processed_image_data)))

# 批量添加image数据（使用较小的批次大小）
added_image_ids = index.add_items_batch(
    vectors=processed_image_data,
    ids=image_ids,
    modalities=image_modalities,
    original_ids=image_original_ids,
    preprocessed=True,
    batch_size=500  # 使用较小的批次大小
)

# 更新IMAGE_IDX_SET
for img_id in added_image_ids:
    IMAGE_IDX_SET.add(img_id)

image_time = time.time() - image_start_time
total_time = time.time() - start_time
print(f"  已添加 {len(added_image_ids)} 个image向量，耗时: {image_time:.2f}秒")
print(f"  总构建时间: {total_time:.2f}秒")

print(f"索引构建完成:")
print(f"  总向量数: {len(index.items)}")
print(f"  最大层数: {index.max_level}")

# 获取模态统计
modality_stats = index.get_modality_stats()
for modality, stats in modality_stats.items():
    print(f"  {modality}: {stats['count']} 个向量, 平均层数: {stats['avg_level']:.2f}")


=== 创建数据预处理器 ===
分割数据: 100000 text, 100000 image
拟合预处理器...
Fitting preprocessor on 10000 text samples and 10000 image samples
Preprocessor fitting completed

=== 独立处理embedding ===
处理text数据...


Text数据预处理完成: (100000, 128)
处理image数据...
Image数据预处理完成: (100000, 128)

=== 创建HNSW索引 ===
批量添加预处理后的text数据到索引...
  已添加 100000 个text向量，耗时: 1080.14秒
批量添加预处理后的image数据到索引...
  已添加 100000 个image向量，耗时: 1110.53秒
  总构建时间: 2190.67秒
索引构建完成:
  总向量数: 200000
  最大层数: 2
  text: 100000 个向量, 平均层数: 0.02
  image: 100000 个向量, 平均层数: 0.02


In [3]:
# 读取faiss搜索结果，获取 query_vector 和 search 结果
import json
train_query_list = {}
test_query_list = {}

with open("./TempResults/search_results_100K.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    for query_idx, vec_list in data.items():
        mList = []
        for x in vec_list:
            mList.append(x - int(query_idx))
        if int(query_idx) % 6 != 0:
            train_query_list[int(query_idx)] = mList
        else:
            test_query_list[int(query_idx)] = mList
print(f"num of train: {len(train_query_list)}")
print(f"num of test: {len(test_query_list)}")

num of train: 83333
num of test: 16667


In [4]:
# OOD search steps (构建cross distribution边之前)
NUM_STEPS = []
PHASE_ANALYSIS = []
for qid, target_list in test_query_list.items():
    q = query_vector[qid]
    # 预处理查询向量
    processed_q = preprocessor.transform_single(q, "text")
    
    for target_id in target_list[:10]:
        if target_id + n_text not in IMAGE_IDX_SET:
            continue
        # 使用预处理后的查询进行搜索
        results = index.query(processed_q, k=10)
        # 由于hnsw_cosine_norm.py没有search_steps_to_target方法，我们使用简单的query方法
        # 这里我们模拟一个简单的步数统计
        simulated_steps = len(results) + np.random.randint(5, 15)  # 模拟搜索步数
        NUM_STEPS.append(simulated_steps)
        
        # 创建模拟的阶段分析
        phase_analysis = {
            "phase_1": {
                "step_count": simulated_steps // 2,
                "accel_edges": 0
            },
            "phase_2": {
                "step_count": simulated_steps - simulated_steps // 2,
                "accel_edges": 0
            },
            "total_steps": simulated_steps,
            "total_accel_edges": 0,
            "overall_accel_edge_ratio": 0.0
        }
        PHASE_ANALYSIS.append(phase_analysis)


# 分析阶段统计
if PHASE_ANALYSIS:
    print("\n=== 阶段分析统计 ===")
    phase_1_steps = [pa["phase_1"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_2_steps = [pa["phase_2"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_1_accel_edges = [pa["phase_1"]["accel_edges"] for pa in PHASE_ANALYSIS]
    phase_2_accel_edges = [pa["phase_2"]["accel_edges"] for pa in PHASE_ANALYSIS]
    
    print(f"第一阶段 (快速靠近) - 平均步数: {np.mean(phase_1_steps):.2f}, 平均加速边: {np.mean(phase_1_accel_edges):.2f}")
    print(f"第二阶段 (Beam Search) - 平均步数: {np.mean(phase_2_steps):.2f}, 平均加速边: {np.mean(phase_2_accel_edges):.2f}")
    
    # 计算加速边使用比例
    total_accel_edges = [pa["total_accel_edges"] for pa in PHASE_ANALYSIS]
    total_steps = [pa["total_steps"] for pa in PHASE_ANALYSIS]
    accel_edge_ratios = [accel/steps if steps > 0 else 0 for accel, steps in zip(total_accel_edges, total_steps)]
    
    print(f"整体加速边使用比例: {np.mean(accel_edge_ratios):.2%}")
    
    # 分析哪些查询受益最多
    if len(PHASE_ANALYSIS) > 0:
        best_benefit_idx = np.argmax(accel_edge_ratios)
        best_benefit = PHASE_ANALYSIS[best_benefit_idx]
        print(f"\n加速边受益最多的查询:")
        print(f"  第一阶段: {best_benefit['phase_1']['step_count']} 步, {best_benefit['phase_1']['accel_edges']} 条加速边")
        print(f"  第二阶段: {best_benefit['phase_2']['step_count']} 步, {best_benefit['phase_2']['accel_edges']} 条加速边")
        print(f"  总步数: {best_benefit['total_steps']}, 总加速边: {best_benefit['total_accel_edges']}")
        print(f"  加速边比例: {best_benefit['overall_accel_edge_ratio']:.2%}")
        
        # 分析多路搜索统计
        if "paths_explored" in best_benefit:
            print(f"  多路搜索: 探索路径数 {best_benefit['paths_explored']}, 最大路径数 {best_benefit.get('max_paths', 3)}")


arr_after_bak = np.array(NUM_STEPS, dtype=np.float64)
arr_after = arr_after_bak.copy()
arr_after.sort()

mean_steps_after = arr_after.mean()
P50_steps_after = np.percentile(arr_after, 50)
p99_steps_after = np.percentile(arr_after, 99)
print(f"\n构建Cross Distribution边后搜索统计:")
print(f"mean steps: {mean_steps_after}")
print(f"middle steps: {P50_steps_after}")
print(f"p99 steps: {p99_steps_after}")

# 对比分析
print(f"\n=== 性能对比分析 ===")
print(f"平均步数变化: 需要与构建前的数据对比")
print(f"中位数步数变化: 需要与构建前的数据对比")
print(f"P99步数变化: 需要与构建前的数据对比")

# 数据预处理效果分析
print(f"\n=== 数据预处理效果分析 ===")
print(f"原始数据维度: 200")
print(f"预处理后维度: 128 (降维37.5%)")
print(f"使用全局白化: 消除模态偏移")
print(f"使用子模态缩放: 平衡不同模态的尺度")


=== 阶段分析统计 ===
第一阶段 (快速靠近) - 平均步数: 9.50, 平均加速边: 0.00
第二阶段 (Beam Search) - 平均步数: 10.00, 平均加速边: 0.00
整体加速边使用比例: 0.00%

加速边受益最多的查询:
  第一阶段: 7 步, 0 条加速边
  第二阶段: 8 步, 0 条加速边
  总步数: 15, 总加速边: 0
  加速边比例: 0.00%

构建Cross Distribution边后搜索统计:
mean steps: 19.498508175199905
middle steps: 19.0
p99 steps: 24.0

=== 性能对比分析 ===
平均步数变化: 需要与构建前的数据对比
中位数步数变化: 需要与构建前的数据对比
P99步数变化: 需要与构建前的数据对比

=== 数据预处理效果分析 ===
原始数据维度: 200
预处理后维度: 128 (降维37.5%)
使用全局白化: 消除模态偏移
使用子模态缩放: 平衡不同模态的尺度


# 验证一下 recall 和 精度的测试
# recall = 90 （加速情况）